In [13]:
#模块
from sklearn import metrics
import pandas as pd
import numpy as np
from scipy.sparse import *
from sklearn.metrics.pairwise import pairwise_distances
import math
import random
import matplotlib.pyplot as plt
import copy
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.neighbors import KNeighborsClassifier 
import time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

def construct_W(X, **kwargs):

    # default metric is 'cosine'
    if 'metric' not in kwargs.keys():
        kwargs['metric'] = 'cosine'

    # default neighbor mode is 'knn' and default neighbor size is 5
    if 'neighbor_mode' not in kwargs.keys():
        kwargs['neighbor_mode'] = 'knn'
    if kwargs['neighbor_mode'] == 'knn' and 'k' not in kwargs.keys():
        kwargs['k'] = 5
    if kwargs['neighbor_mode'] == 'supervised' and 'k' not in kwargs.keys():
        kwargs['k'] = 5
    if kwargs['neighbor_mode'] == 'supervised' and 'y' not in kwargs.keys():
        print ('Warning: label is required in the supervised neighborMode!!!')
        exit(0)

    # default weight mode is 'binary', default t in heat kernel mode is 1
    if 'weight_mode' not in kwargs.keys():
        kwargs['weight_mode'] = 'binary'
    if kwargs['weight_mode'] == 'heat_kernel':
        if kwargs['metric'] != 'euclidean':
            kwargs['metric'] = 'euclidean'
        if 't' not in kwargs.keys():
            kwargs['t'] = 1
    elif kwargs['weight_mode'] == 'cosine':
        if kwargs['metric'] != 'cosine':
            kwargs['metric'] = 'cosine'

    # default fisher_score and reliefF mode are 'false'
    if 'fisher_score' not in kwargs.keys():
        kwargs['fisher_score'] = False
    if 'reliefF' not in kwargs.keys():
        kwargs['reliefF'] = False

    n_samples, n_features = np.shape(X)

    # choose 'knn' neighbor mode
    if kwargs['neighbor_mode'] == 'knn':
        k = kwargs['k']
        if kwargs['weight_mode'] == 'binary':
            if kwargs['metric'] == 'euclidean':
                # compute pairwise euclidean distances
                D = pairwise_distances(X)
                D **= 2
                # sort the distance matrix D in ascending order
                dump = np.sort(D, axis=1)
                idx = np.argsort(D, axis=1)
                # choose the k-nearest neighbors for each instance
                idx_new = idx[:, 0:k+1]
                G = np.zeros((n_samples*(k+1), 3))
                G[:, 0] = np.tile(np.arange(n_samples), (k+1, 1)).reshape(-1)
                G[:, 1] = np.ravel(idx_new, order='F')
                G[:, 2] = 1
                # build the sparse affinity matrix W
                W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
                bigger = np.transpose(W) > W
                W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
                return W

            elif kwargs['metric'] == 'cosine':
                # normalize the data first
                X_normalized = np.power(np.sum(X*X, axis=1), 0.5)
                for i in range(n_samples):
                    X[i, :] = X[i, :]/max(1e-12, X_normalized[i])
                # compute pairwise cosine distances
                D_cosine = np.dot(X, np.transpose(X))
                # sort the distance matrix D in descending order
                dump = np.sort(-D_cosine, axis=1)
                idx = np.argsort(-D_cosine, axis=1)
                idx_new = idx[:, 0:k+1]
                G = np.zeros((n_samples*(k+1), 3))
                G[:, 0] = np.tile(np.arange(n_samples), (k+1, 1)).reshape(-1)
                G[:, 1] = np.ravel(idx_new, order='F')
                G[:, 2] = 1
                # build the sparse affinity matrix W
                W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
                bigger = np.transpose(W) > W
                W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
                return W

        elif kwargs['weight_mode'] == 'heat_kernel':
            t = kwargs['t']
            # compute pairwise euclidean distances
            D = pairwise_distances(X)
            D **= 2
            # sort the distance matrix D in ascending order
            dump = np.sort(D, axis=1)
            idx = np.argsort(D, axis=1)
            idx_new = idx[:, 0:k+1]
            dump_new = dump[:, 0:k+1]
            # compute the pairwise heat kernel distances
            dump_heat_kernel = np.exp(-dump_new/(2*t*t))
            G = np.zeros((n_samples*(k+1), 3))
            G[:, 0] = np.tile(np.arange(n_samples), (k+1, 1)).reshape(-1)
            G[:, 1] = np.ravel(idx_new, order='F')
            G[:, 2] = np.ravel(dump_heat_kernel, order='F')
            # build the sparse affinity matrix W
            W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W) > W
            W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
            return W

        elif kwargs['weight_mode'] == 'cosine':
            # normalize the data first
            X_normalized = np.power(np.sum(X*X, axis=1), 0.5)
            for i in range(n_samples):
                    X[i, :] = X[i, :]/max(1e-12, X_normalized[i])
            # compute pairwise cosine distances
            D_cosine = np.dot(X, np.transpose(X))
            # sort the distance matrix D in ascending order
            dump = np.sort(-D_cosine, axis=1)
            idx = np.argsort(-D_cosine, axis=1)
            idx_new = idx[:, 0:k+1]
            dump_new = -dump[:, 0:k+1]
            G = np.zeros((n_samples*(k+1), 3))
            G[:, 0] = np.tile(np.arange(n_samples), (k+1, 1)).reshape(-1)
            G[:, 1] = np.ravel(idx_new, order='F')
            G[:, 2] = np.ravel(dump_new, order='F')
            # build the sparse affinity matrix W
            W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W) > W
            W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
            return W

    # choose supervised neighborMode
    elif kwargs['neighbor_mode'] == 'supervised':
        k = kwargs['k']
        # get true labels and the number of classes
        y = kwargs['y']
        label = np.unique(y)
        n_classes = np.unique(y).size
        # construct the weight matrix W in a fisherScore way, W_ij = 1/n_l if yi = yj = l, otherwise W_ij = 0
        if kwargs['fisher_score'] is True:
            W = lil_matrix((n_samples, n_samples))
            for i in range(n_classes):
                class_idx = (y == label[i])
                class_idx_all = (class_idx[:, np.newaxis] & class_idx[np.newaxis, :])
                W[class_idx_all] = 1.0/np.sum(np.sum(class_idx))
            return W

        # construct the weight matrix W in a reliefF way, NH(x) and NM(x,y) denotes a set of k nearest
        # points to x with the same class as x, a different class (the class y), respectively. W_ij = 1 if i = j;
        # W_ij = 1/k if x_j \in NH(x_i); W_ij = -1/(c-1)k if x_j \in NM(x_i, y)
        if kwargs['reliefF'] is True:
            # when xj in NH(xi)
            G = np.zeros((n_samples*(k+1), 3))
            id_now = 0
            for i in range(n_classes):
                class_idx = np.column_stack(np.where(y == label[i]))[:, 0]
                D = pairwise_distances(X[class_idx, :])
                D **= 2
                idx = np.argsort(D, axis=1)
                idx_new = idx[:, 0:k+1]
                n_smp_class = (class_idx[idx_new[:]]).size
                if len(class_idx) <= k:
                    k = len(class_idx) - 1
                G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx, (k+1, 1)).reshape(-1)
                G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx[idx_new[:]], order='F')
                G[id_now:n_smp_class+id_now, 2] = 1.0/k
                id_now += n_smp_class
            W1 = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            # when i = j, W_ij = 1
            for i in range(n_samples):
                W1[i, i] = 1
            # when x_j in NM(x_i, y)
            G = np.zeros((n_samples*k*(n_classes - 1), 3))
            id_now = 0
            for i in range(n_classes):
                class_idx1 = np.column_stack(np.where(y == label[i]))[:, 0]
                X1 = X[class_idx1, :]
                for j in range(n_classes):
                    if label[j] != label[i]:
                        class_idx2 = np.column_stack(np.where(y == label[j]))[:, 0]
                        X2 = X[class_idx2, :]
                        D = pairwise_distances(X1, X2)
                        idx = np.argsort(D, axis=1)
                        idx_new = idx[:, 0:k]
                        n_smp_class = len(class_idx1)*k
                        G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx1, (k, 1)).reshape(-1)
                        G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx2[idx_new[:]], order='F')
                        G[id_now:n_smp_class+id_now, 2] = -1.0/((n_classes-1)*k)
                        id_now += n_smp_class
            W2 = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W2) > W2
            W2 = W2 - W2.multiply(bigger) + np.transpose(W2).multiply(bigger)
            W = W1 + W2
            return W

        if kwargs['weight_mode'] == 'binary':
            if kwargs['metric'] == 'euclidean':
                G = np.zeros((n_samples*(k+1), 3))
                id_now = 0
                for i in range(n_classes):
                    class_idx = np.column_stack(np.where(y == label[i]))[:, 0]
                    # compute pairwise euclidean distances for instances in class i
                    D = pairwise_distances(X[class_idx, :])
                    D **= 2
                    # sort the distance matrix D in ascending order for instances in class i
                    idx = np.argsort(D, axis=1)
                    idx_new = idx[:, 0:k+1]
                    n_smp_class = len(class_idx)*(k+1)
                    G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx, (k+1, 1)).reshape(-1)
                    G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx[idx_new[:]], order='F')
                    G[id_now:n_smp_class+id_now, 2] = 1
                    id_now += n_smp_class
                # build the sparse affinity matrix W
                W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
                bigger = np.transpose(W) > W
                W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
                return W

            if kwargs['metric'] == 'cosine':
                # normalize the data first
                X_normalized = np.power(np.sum(X*X, axis=1), 0.5)
                for i in range(n_samples):
                    X[i, :] = X[i, :]/max(1e-12, X_normalized[i])
                G = np.zeros((n_samples*(k+1), 3))
                id_now = 0
                for i in range(n_classes):
                    class_idx = np.column_stack(np.where(y == label[i]))[:, 0]
                    # compute pairwise cosine distances for instances in class i
                    D_cosine = np.dot(X[class_idx, :], np.transpose(X[class_idx, :]))
                    # sort the distance matrix D in descending order for instances in class i
                    idx = np.argsort(-D_cosine, axis=1)
                    idx_new = idx[:, 0:k+1]
                    n_smp_class = len(class_idx)*(k+1)
                    G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx, (k+1, 1)).reshape(-1)
                    G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx[idx_new[:]], order='F')
                    G[id_now:n_smp_class+id_now, 2] = 1
                    id_now += n_smp_class
                # build the sparse affinity matrix W
                W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
                bigger = np.transpose(W) > W
                W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
                return W

        elif kwargs['weight_mode'] == 'heat_kernel':
            G = np.zeros((n_samples*(k+1), 3))
            id_now = 0
            for i in range(n_classes):
                class_idx = np.column_stack(np.where(y == label[i]))[:, 0]
                # compute pairwise cosine distances for instances in class i
                D = pairwise_distances(X[class_idx, :])
                D **= 2
                # sort the distance matrix D in ascending order for instances in class i
                dump = np.sort(D, axis=1)
                idx = np.argsort(D, axis=1)
                idx_new = idx[:, 0:k+1]
                dump_new = dump[:, 0:k+1]
                t = kwargs['t']
                # compute pairwise heat kernel distances for instances in class i
                dump_heat_kernel = np.exp(-dump_new/(2*t*t))
                n_smp_class = len(class_idx)*(k+1)
                G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx, (k+1, 1)).reshape(-1)
                G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx[idx_new[:]], order='F')
                G[id_now:n_smp_class+id_now, 2] = np.ravel(dump_heat_kernel, order='F')
                id_now += n_smp_class
            # build the sparse affinity matrix W
            W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W) > W
            W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
            return W

        elif kwargs['weight_mode'] == 'cosine':
            # normalize the data first
            X_normalized = np.power(np.sum(X*X, axis=1), 0.5)
            for i in range(n_samples):
                X[i, :] = X[i, :]/max(1e-12, X_normalized[i])
            G = np.zeros((n_samples*(k+1), 3))
            id_now = 0
            for i in range(n_classes):
                class_idx = np.column_stack(np.where(y == label[i]))[:, 0]
                # compute pairwise cosine distances for instances in class i
                D_cosine = np.dot(X[class_idx, :], np.transpose(X[class_idx, :]))
                # sort the distance matrix D in descending order for instances in class i
                dump = np.sort(-D_cosine, axis=1)
                idx = np.argsort(-D_cosine, axis=1)
                idx_new = idx[:, 0:k+1]
                dump_new = -dump[:, 0:k+1]
                n_smp_class = len(class_idx)*(k+1)
                G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx, (k+1, 1)).reshape(-1)
                G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx[idx_new[:]], order='F')
                G[id_now:n_smp_class+id_now, 2] = np.ravel(dump_new, order='F')
                id_now += n_smp_class
            # build the sparse affinity matrix W
            W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W) > W
            W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
            return W


In [9]:
#函数
import numpy as np
from scipy.sparse import *
res = pd.DataFrame(0,columns=["spnsi","speci","GM","f1","AUC"],index=dataList)
def fisher_score(X, y):

    # Construct weight matrix W in a fisherScore way
    kwargs = {"neighbor_mode": "supervised", "fisher_score": True, 'y': y}
    W = construct_W(X, **kwargs)

    # build the diagonal D matrix from affinity matrix W
    D = np.array(W.sum(axis=1))
    L = W
    tmp = np.dot(np.transpose(D), X)
    D = diags(np.transpose(D), [0])
    Xt = np.transpose(X)
    t1 = np.transpose(np.dot(Xt, D.todense()))
    t2 = np.transpose(np.dot(Xt, L.todense()))
    # compute the numerator of Lr
    D_prime = np.sum(np.multiply(t1, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # compute the denominator of Lr
    L_prime = np.sum(np.multiply(t2, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # avoid the denominator of Lr to be 0
    D_prime[D_prime < 1e-12] = 10000
    lap_score = 1 - np.array(np.multiply(L_prime, 1/D_prime))[0, :]

    # compute fisher score from laplacian score, where fisher_score = 1/lap_score - 1
    score = 1.0/lap_score - 1
    return np.transpose(score)

def feature_ranking(score):
    """
    Rank features in descending order according to fisher score, the larger the fisher score, the more important the
    feature is
    """
    idx = np.argsort(score, 0)
    return idx[::-1]

def GM_score(x,y): #计算GM 传入numpy
    cm1 = confusion_matrix(x,y)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    return sensitivity1,specificity1

def getAncestors(feat):
    global relativeList
    for relative in DAG.columns:
        if(DAG.loc[relative,feat] == 1):
            relativeList.append(relative)
            getAncestors(relative)
def getDescendants(feat):
    global relativeList
    for relative in DAG.columns:
        if(DAG.loc[feat,relative] == 1):
            relativeList.append(relative)
            getAncestors(relative)
def get(feat):
    global relativeList
    corr = []
    #先找到当前节点的所有祖先或后代节点
    relativeList = []
    getDescendants(feat)
    getAncestors(feat)
    relativeSet = set(relativeList)
    for relative in relativeSet:
        corr.append(metrics.normalized_mutual_info_score(data.loc[:,c1],data.loc[:,c2]))
    #print(corr)
    if(corr == []):
        return 0
    else:
        return np.mean(corr)

In [26]:
# dataList = ["MM/MM-BP.csv","MM/MM-CC.csv","MM/MM-MF.csv","MM/MM-BPCC.csv",
#             "MM/MM-BPMF.csv","MM/MM-CCMF.csv","MM/MM-BPCCMF.csv",
#            "CE/CE-BP.csv","CE/CE-CC.csv","CE/CE-MF.csv","CE/CE-BPCC.csv",
#             "CE/CE-BPMF.csv","CE/CE-CCMF.csv","CE/CE-BPCCMF.csv",
#            "DM/DM-BP.csv","DM/DM-CC.csv","DM/DM-MF.csv","DM/DM-BPCC.csv",
#             "DM/DM-BPMF.csv","DM/DM-CCMF.csv","DM/DM-BPCCMF.csv",
#            "SC/SC-BP.csv","SC/SC-CC.csv","SC/SC-MF.csv","SC/SC-BPCC.csv",
#             "SC/SC-BPMF.csv","SC/SC-CCMF.csv","SC/SC-BPCCMF.csv"]
dataList = ["Cities.csv","NYDailyheadings.csv","SportsTweetsc.csv","SportsTweetst.csv","Stumbleupon.csv"]
dataName = "SportsTweetst.csv"

In [27]:
#加载数据 预处理
time_start = time.time()  # 记录开始时间
data = pd.read_csv("../data/real_world_datasets/Datasets/"+dataName,index_col=0)
if(dataName == "Cities.csv"):
    data["longevity influence"] = data["longevity influence"].apply(lambda x : 1 if x > 1.751738e+09 else 0)
label = data["longevity influence"]
#删除无用列
data = data.drop(["longevity influence"],axis=1)
#过滤低纬度特征
for c in data.columns:
    data[c] = data[c].astype('int')
    if(data[c].sum() < 3):
        data.drop(c,axis=1,inplace=True)
DAG = pd.read_csv("../data/real_world_datasets/Hierarchy/"+dataName,index_col=0)
DAG = DAG.loc[data.columns,data.columns]
labels_pred=data.values.tolist()
labels_true=label.values.tolist()
#读取计算结果
hierarchicalR = pd.read_csv("../"+dataName,index_col=0)
# Relevance = pd.Series(fisher_score(labels_pred,labels_true),index=data.columns)
#α惩罚项
# SUMetrics = pd.DataFrame(0,columns=data.columns,index=data.columns)
# for c1 in data.columns:
#     print(c1)
#     for c2 in data.columns:
#         if(c1 != c2):
#             SUMetrics.loc[c1,c2] = metrics.normalized_mutual_info_score(data.loc[:,c1],data.loc[:,c2])

# relativeList = []
# global relativeList
# avgCorr = pd.Series(0,index=data.columns)
# for c in data.columns:
#     print(1)
#     avgCorr.loc[c] = get(c)

#β惩罚项

# relativeList = []
# global relativeList
# countAncestors = pd.Series(0,index=data.columns)
# for c in data.columns[4:]:
#     print(1)
#     relativeList = []
#     getAncestors(c) 
#     countAncestors.loc[c] = len(set(relativeList))

#归一化
# Relevance = (Relevance - Relevance.min()) / (Relevance.max() - Relevance.min())
# avgCorr = (avgCorr - avgCorr.min()) / (avgCorr.max() - avgCorr.min())
# countAncestors = (countAncestors - countAncestors.min()) / (countAncestors.max() - countAncestors.min())
# #带层次冗余惩罚的相关性系数
# alpha = 0.1
# beta = 0.2
#hierarchicalR = Relevance - alpha * avgCorr - beta * countAncestors
#hierarchicalR = hierarchicalR.sort_values(ascending = False)

In [30]:
#执行
num = int(data.shape[1]*0.7)
sensitivity = []
specificity = []
F1 = []
AUC = []
numList = []
#10折交叉验证
kf = KFold(n_splits=5,shuffle=True)
for train_index ,test_index in kf.split(data):
    trainData = data.iloc[train_index,:]
    testData = data.iloc[test_index,:]
    Y_train = label.values[train_index] #用于训练模型
    Y_test = label.values[test_index]#用于交叉验证

    selectFeatures = hierarchicalR[:num].index#CFS
    #selectFeatures = data.columns

    X_train = trainData.loc[:,selectFeatures]
    X_test = testData.loc[:,selectFeatures]
#     gnb = GaussianNB()
#     predictList = gnb.fit(X_train, Y_train).predict(X_test)
#     from sklearn.svm import SVC
#     clf = make_pipeline(StandardScaler(), SVC(gamma=0.5))
#     clf.fit(X_train, Y_train)
#     predictList = clf.predict(X_test)
    rfc = RandomForestClassifier(random_state=0)
    rfc = rfc.fit(X_train,Y_train)
    predictList = rfc.predict(X_test)
    print(predictList)
    print(Y_test)
    sensi,speci= GM_score(predictList,Y_test)
    sensitivity.append(sensi)
    specificity.append(speci)
    F1.append(f1_score(np.array(predictList),Y_test))
    try:
        AUC.append(roc_auc_score(np.array(predictList),Y_test))
    except ValueError:
        pass
#standard
a = np.nanmean(sensitivity)
b = np.nanmean(specificity)
print(a)
print(b)
print("GM")
print(round(math.sqrt(a*b)*100,1))
print("f1")
print(round(np.nanmean(F1)*100,1))
print("AUC") 
print(round(np.nanmean(AUC)*100,1))
res.loc[dataName,"spnsi"] = round(a*100,1)
res.loc[dataName,"speci"] = round(b*100,1)
res.loc[dataName,"GM"] = round(math.sqrt(a*b)*100,1)
res.loc[dataName,"f1"] = round(np.nanmean(F1)*100,1)
res.loc[dataName,"AUC"] = round(np.nanmean(AUC)*100,1)

time_end = time.time()  # 记录结束时间
time_sum = time_end - time_start  # 计算的时间差为程序的执行时间，单位为秒/s
print(time_sum)

[0 0 0 1 0 1 1 1 1 1 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 0 1 1 1 1 0 0 1 1 1 0 1
 1 0 1 1 1 1 0 0 0 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1
 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1
 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0

In [35]:
res.to_csv("../res.csv")

In [2813]:
label = pd.read_csv("data/real_world_datasets/Datasets/"+dataName,index_col=0)["longevity influence"]

In [2815]:
label = label.apply(lambda x : 0 if x < 1e+09 else 1)

In [2816]:
label.value_counts()

1    173
0     39
Name: longevity influence, dtype: int64

In [22]:
hierarchicalR = hierarchicalR-hierarchicalR.mean()
hierarchicalR = hierarchicalR.sort_values(ascending = False)
hierarchicalR.to_csv("../Tweetsc.csv")

In [21]:
hierarchicalR.mean()

0.0077138798539807055